In [55]:
import numpy as np

def normalize(pc):
    pc = pc - pc.mean(axis=0)
    scale = np.max(np.linalg.norm(pc, axis=1))
    return pc / scale

def box_occlusion(pc, axis='z', center=0.2, width=0.2):
    """
    沿 axis 在给定 center ± width/2 范围内遮挡一块区域
    返回未被遮挡的点
    """
    axis_idx = {'x': 0, 'y': 1, 'z': 2}[axis]
    lower = center - width / 2
    upper = center + width / 2
    visible = pc[(pc[:, axis_idx] < lower) | (pc[:, axis_idx] > upper)]
    return visible

def occlude_and_sample(pc, axis='z', center=0.2, width=0.2, target_points=1024):
    visible = box_occlusion(normalize(pc), axis, center, width)
    N = visible.shape[0]
    if N >= target_points:
        idx = np.random.choice(N, target_points, replace=False)
        return visible[idx]
    else:
        idx = np.random.choice(N, target_points, replace=True)
        return visible[idx]



if __name__ == "__main__":
    X_train = np.load("train_points.npy")
    X_test = np.load("test_points.npy")

    partial_X_train = np.array([occlude_and_sample(pc) for pc in X_train])
    partial_X_test = np.array([occlude_and_sample(pc) for pc in X_test])


    np.save("partial_train.npy", partial_X_train)
    np.save("partial_test.npy", partial_X_test)


In [57]:
import open3d as o3d
X_full = np.load("train_points.npy")

# 随机选一个样本进行可视化
i = 10
pc_full = normalize(X_full[i])
pc_partial = occlude_and_sample(X_full[i], axis='z', center=0.0, width=0.2)

# Open3D 可视化
pcd_full = o3d.geometry.PointCloud()
pcd_full.points = o3d.utility.Vector3dVector(pc_full)
pcd_full.paint_uniform_color([0, 1, 0])  # green: full

pcd_partial = o3d.geometry.PointCloud()
pcd_partial.points = o3d.utility.Vector3dVector(pc_partial)
pcd_partial.paint_uniform_color([1, 0, 0])  # red: partial

# 可视化平移以分开显示
o3d.visualization.draw_geometries([
    pcd_partial.translate((0.5, 0, 0)),
    pcd_full.translate((-0.5, 0, 0))
])

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNetEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 128, 1)
        self.conv2 = nn.Conv1d(128, 256, 1)
        self.conv3 = nn.Conv1d(256, 1024, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):
        x = x.transpose(2, 1)  # [B, 3, N]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2)[0]  # global max pooling [B, 1024]
        return x

class FoldingDecoder(nn.Module):
    def __init__(self, coarse_points=1024, grid_size=2):
        super().__init__()
        self.grid_size = grid_size
        self.folding_mlp = nn.Sequential(
            nn.Linear(1029, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

    def build_grid(self, B, device):
        # 构造 [grid_size × grid_size, 2] 的网格
        range_ = torch.linspace(-0.05, 0.05, self.grid_size, device=device)
        meshgrid = torch.meshgrid(range_, range_, indexing='ij')  # 使用 'ij' 避免 future warning
        grid = torch.stack(meshgrid, dim=-1).reshape(-1, 2)  # [16, 2]
        grid = grid.unsqueeze(0).unsqueeze(0)  # [1, 1, 16, 2]
        grid = grid.repeat(B, 1024, 1, 1)      # [B, 1024, 16, 2]
        return grid

    def forward(self, coarse, global_feat):
        B = coarse.shape[0]
        grid = self.build_grid(B, coarse.device)  # [B, 1024, 16, 2]
        
        coarse = coarse.unsqueeze(2).repeat(1, 1, self.grid_size ** 2, 1)  # [B, 1024, 16, 3]
        feat = global_feat.unsqueeze(1).unsqueeze(2).repeat(1, 1024, self.grid_size ** 2, 1)  # [B, 1024, 16, 1024]

        concat = torch.cat([grid, coarse, feat], dim=-1)  # [B, 1024, 16, 1029]
        fine = self.folding_mlp(concat).view(B, -1, 3)    # [B, 1024×16, 3] = [B, 4096, 3]
        return fine


class PCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = PointNetEncoder()
        self.coarse_fc = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024 * 3)
        )
        self.decoder = FoldingDecoder()

    def forward(self, x):
        feat = self.encoder(x)              # [B, 1024]
        coarse = self.coarse_fc(feat).view(-1, 1024, 3)  # [B, 1024, 3]
        fine = self.decoder(coarse, feat)   # [B, 4096, 3]
        return coarse, fine


In [59]:
def chamfer_distance(pcd1, pcd2):
    x, y = pcd1, pcd2
    x = x.unsqueeze(2)  # [B, N, 1, 3]
    y = y.unsqueeze(1)  # [B, 1, M, 3]
    dist = torch.norm(x - y, dim=-1)  # [B, N, M]
    dist1 = torch.min(dist, dim=2)[0]
    dist2 = torch.min(dist, dim=1)[0]
    return torch.mean(dist1) + torch.mean(dist2)


In [60]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

partial = torch.tensor(np.load("partial_train.npy"), dtype=torch.float32)
complete = torch.tensor(np.load("train_points.npy"), dtype=torch.float32)

train_loader = DataLoader(TensorDataset(partial, complete), batch_size=4, shuffle=True)

model = PCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [61]:
for epoch in range(30):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        _, fine = model(x)
        loss = chamfer_distance(fine, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}: Loss {total_loss / len(train_loader):.4f}")
torch.save(model.state_dict(), "pcn_model.pth")

Epoch 0: Loss 0.1693
Epoch 1: Loss 0.1230
Epoch 2: Loss 0.1122
Epoch 3: Loss 0.1064
Epoch 4: Loss 0.1015
Epoch 5: Loss 0.0997
Epoch 6: Loss 0.0977
Epoch 7: Loss 0.0962
Epoch 8: Loss 0.0950
Epoch 9: Loss 0.0935
Epoch 10: Loss 0.0925
Epoch 11: Loss 0.0918
Epoch 12: Loss 0.0902
Epoch 13: Loss 0.0899
Epoch 14: Loss 0.0895
Epoch 15: Loss 0.0888
Epoch 16: Loss 0.0881
Epoch 17: Loss 0.0877
Epoch 18: Loss 0.0870
Epoch 19: Loss 0.0870
Epoch 20: Loss 0.0860
Epoch 21: Loss 0.0859
Epoch 22: Loss 0.0852
Epoch 23: Loss 0.0853
Epoch 24: Loss 0.0847
Epoch 25: Loss 0.0840
Epoch 26: Loss 0.0842
Epoch 27: Loss 0.0838
Epoch 28: Loss 0.0833
Epoch 29: Loss 0.0834


In [62]:

model = PCN().to(device)
model.load_state_dict(torch.load("pcn_model.pth", weights_only=True))

model.eval()


PCN(
  (encoder): PointNetEncoder(
    (conv1): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (coarse_fc): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=3072, bias=True)
  )
  (decoder): FoldingDecoder(
    (folding_mlp): Sequential(
      (0): Linear(in_features=1029, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)

In [ ]:
import open3d as o3d
# 加载测试数据（根据你的数据改成 train/test 都可以）
partial = torch.tensor(np.load("partial_train.npy"), dtype=torch.float32).to(device)
complete = torch.tensor(np.load("train_points.npy"), dtype=torch.float32).to(device)

# 可视化函数
def show_point_cloud(pc, color=[0.5, 0.5, 0.5]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc)
    pcd.paint_uniform_color(color)
    return pcd

# 可视化第 i 个样本
def visualize_sample(i=0):
    input_i = partial[i].unsqueeze(0)  # 加 batch 维度
    gt_i = complete[i].unsqueeze(0)    # gWround truth

    with torch.no_grad():
        _, pred_i = model(input_i)

    # 转 numpy
    input_np = input_i[0].cpu().numpy()
    pred_np = pred_i[0].cpu().numpy()
    gt_np = gt_i[0].cpu().numpy()

    # 显示三者：红=partial，蓝=pred，绿=GT
    input_pcd = show_point_cloud(input_np, [1, 0, 0])
    pred_pcd  = show_point_cloud(pred_np,  [0, 0, 1])
    gt_pcd    = show_point_cloud(gt_np,    [0, 1, 0])

    o3d.visualization.draw_geometries(
        [input_pcd.translate((1.0, 0, 0)), 
         pred_pcd.translate((0, 0, 0)),
         gt_pcd.translate((-1.0, 0, 0))],
        window_name=f"Sample {i}: Red=Input, Blue=Pred, Green=GT"
    )

    # Chamfer Distance 评估
    cd = chamfer_distance(pred_i, gt_i).item()
    print(f"Sample {i} Chamfer Distance: {cd:.6f}")

# 调用可视化函数（可以改索引）
visualize_sample(i=49)

Sample 49 Chamfer Distance: 0.088589


In [ ]:
cd_list = []
model.eval()
with torch.no_grad():
    for i in range(len(partial)):
        input_i = partial[i].unsqueeze(0)
        gt_i = complete[i].unsqueeze(0)
        _, pred_i = model(input_i)
        cd = chamfer_distance(pred_i, gt_i).item()
        cd_list.append(cd)

mean_cd = np.mean(cd_list)
std_cd = np.std(cd_list)
print(f"Mean Chamfer Distance: {mean_cd:.6f} ± {std_cd:.6f}")
